# COPD Hospitalisations | Processing

The main tasks completed to clean and preprocess this dataset were:

**Data Manipulation**
1. Rename columns.
2. Reformat 'financial year' values from XX/YY to XXXX/YYYY
3. Remove 'LHD' from LHD name values.
4. Remove 'All' data (Representing a state-wide average).
5. Remove seperate age groups, keeping only rows with "All Ages"
6. Remove columns holding Confidence Interval data.
7. Remove rows holding 'Persons' data in the sex column (Representing a genderless rate per 100,000).
8. Remove 'risk group' column.

## Set Up

Ensure that the required libraries are available by running the below code in the terminal before execution:
- pip install pandas


Execute the following in the jupyter notebook before execution to ensure that the required libraries are imported:

In [1]:
import pandas as pd

## Load Dataset

In [2]:
# File path.
file_path = 'data-raw.csv'

# Read the file.
df = pd.read_csv(file_path)

df

,Risk group,Sex,LHD,Period,"Rate per 100,000 population",LL 95% CI,UL 95% CI
0,35-64 years,Males,Sydney LHD,01/02,119.6,98.1,144.3
1,35-64 years,Males,Sydney LHD,02/03,126.9,104.7,152.2
2,35-64 years,Males,Sydney LHD,03/04,127.4,105.3,152.6
3,35-64 years,Males,Sydney LHD,04/05,151.4,127.5,178.4
4,35-64 years,Males,Sydney LHD,05/06,125.1,103.9,149.4
...,...,...,...,...,...,...,...
6055,All ages,Persons,All LHDs,18/19,225.8,222.8,228.8
6056,All ages,Persons,All LHDs,19/20,205.1,202.3,207.9
6057,All ages,Persons,All LHDs,20/21,145.6,143.3,148
6058,All ages,Persons,All LHDs,21/22,133.4,131.2,135.6


## Data Manipulation

Rename columns to match Air Quality data set.

In [3]:
# Rename columns.
df = df.rename(columns={
    'LHD': 'lhd',
    'Period': 'financial year'
})

# Set column names to lower case.
df.columns = df.columns.str.lower()

Make Year data in 'financial year' column more verbose.

In [4]:
# Reformat 'financial year' values from XX/YY to XXXX/YYYY.
df['financial year'] = df['financial year'].apply(
    lambda x: f'20{x[:2]}/20{x[3:]}' if isinstance(x, str) else x
)

Remove LHD from Local Area Districts values.

In [5]:
# Remove ' LHD' from the 'lhd' column.
df['lhd'] = df['lhd'].str.replace(' LHD', '')

Remove rows representing state-wide aggregated date.

In [7]:
# Remove rows with NaN in the 'lhd' column.
df = df.dropna(subset=['lhd'])

# Remove rows with 'All' in the 'lhd' column.
df = df[~df['lhd'].str.contains('All')]

Remove rows representing an aggregate of all risk groups within a Local Health District.

In [9]:
# Remove rows with NaN in the 'risk group' column.
df = df.dropna(subset=['risk group'])

# Remove rows with 'All ages' in the 'risk group' column.
df = df[df['risk group'].str.contains('All ages')]

df

,risk group,sex,lhd,financial year,"rate per 100,000 population",ll 95% ci,ul 95% ci
2020,All ages,Males,Sydney,2001/2002,313.6,289.8,338.8
2021,All ages,Males,Sydney,2002/2003,282.3,259.4,306.5
2022,All ages,Males,Sydney,2003/2004,298.8,274.3,324.7
2023,All ages,Males,Sydney,2004/2005,326.7,301.3,353.5
2024,All ages,Males,Sydney,2005/2006,283.7,261.6,307.2
...,...,...,...,...,...,...,...
6033,All ages,Persons,Far West,2017/2018,486,421,559.3
6034,All ages,Persons,Far West,2018/2019,479.9,414.8,553.4
6035,All ages,Persons,Far West,2019/2020,425.6,365.9,493.5
6036,All ages,Persons,Far West,2020/2021,351,296.1,414.1


Remove rows holding Confidence Interval data.

In [10]:
# Drop columns with '% ci' in the header
df = df.loc[:, ~df.columns.str.contains('% ci')]

Remove rows holding "Persons" data in the sex column.

In [11]:
# Drop rows with 'Persons' in the 'sex' column.
df = df[~df['sex'].str.contains('Persons')]

Remove the "Risk Group" column

In [12]:
# Drop the 'risk group' column.
df = df.drop(columns=['risk group'])

Drop rows where column titles had been entered as data.

In [13]:
# Drop rows where column title had been entered as data.
df = df[~df['sex'].str.contains('Sex')]

## Output Processed Dataset

In [14]:
# File path.
file_path_output = 'data-processed.csv'

# Save the file.
df.to_csv(file_path_output, index=False)

## View Dataset

In [15]:
df

,sex,lhd,financial year,"rate per 100,000 population"
2020,Males,Sydney,2001/2002,313.6
2021,Males,Sydney,2002/2003,282.3
2022,Males,Sydney,2003/2004,298.8
2023,Males,Sydney,2004/2005,326.7
2024,Males,Sydney,2005/2006,283.7
...,...,...,...,...
5697,Females,Far West,2017/2018,450.9
5698,Females,Far West,2018/2019,408
5699,Females,Far West,2019/2020,413.9
5700,Females,Far West,2020/2021,242.2
